Deep Learning Models -- A collection of various deep learning architectures, models, and tips for TensorFlow and PyTorch in Jupyter Notebooks.
- Author: Sebastian Raschka
- GitHub Repository: https://github.com/rasbt/deeplearning-models

In [ ]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

- Runs on CPU (not recommended here) or GPU (if available)

# Model Zoo -- Convolutional Autoencoder with Nearest-neighbor Interpolation (Trained on 10 categories of the Quickdraw dataset)

A convolutional autoencoder using nearest neighbor upscaling layers that compresses 768-pixel Quickdraw images down to a 7x7x8 (392 pixel) representation.

## Imports

In [4]:
import os
import time

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from glob import iglob, glob
from tqdm import tqdm
from pathlib import Path
import pickle as pkl
import torchsnooper
from torch import nn, optim
from torch.utils import data
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import SubsetRandomSampler, SequentialSampler
from glob import iglob, glob
from matplotlib import pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from tqdm import tqdm
from pathlib import Path

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Dataset

### Create a Custom Data Loader

In [5]:
class ConvEncoderDatasetRAM(data.Dataset):
    '''
    Convolutional Auto encoder dataset.
    '''

    def __init__(self, datadir):
        '''
        Initialization.

        Args:
            datadir: directory of serialized tensors
        '''
        self.paths = glob(datadir + '/*.pkl')
        self.length = len(self.paths)

        # load all tensor into RAM
        tensors = []
        for path in tqdm(self.paths, total=self.length, ascii=True):
            tensor = torch.load(path).numpy()
            tensors.append(tensor)

        tensors = np.array(tensors).astype('float32')
        self.tensors = tensors
        # numpy image to pytorch image need to swap axes
        self.tensors = np.swapaxes(self.tensors, 1, 3)
        print(f'Conv Autoencoder data shape: {tensors.shape}')

    def __len__(self):
        '''
        Denotes the total number of samples
        '''
        return self.length - 1

    def __getitem__(self, index):
        '''
        Generates one sample of data
        '''
        # Load data and get label
        X = self.tensors[index]
        X = torch.from_numpy(X)
        y = self.tensors[index]
        # y = y.reshape(1, -1)
        y = torch.from_numpy(y)
        # print(f'X.shape -> {X.shape} || y.shape -> {y.shape}')
        return X, y

In [6]:
data_dir = r'J:\gspn\data\processed\pnf\2018\15min\tensors'
data_set = ConvEncoderDatasetRAM(data_dir)

# split dataset for training and validation
num_train = len(data_set)
indices = list(range(num_train))
split = int(np.floor(0.8 * num_train))  # hard coded to 0.8

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SequentialSampler(train_idx)
valid_sampler = SequentialSampler(valid_idx)

train_loader = DataLoader(data_set, sampler=train_sampler, batch_size=batch_size, num_workers=0, drop_last=True)
valid_loader = DataLoader(data_set, sampler=valid_sampler, batch_size=batch_size, num_workers=0, drop_last=True)

100%|##########################################################################| 35040/35040 [00:12<00:00, 2792.92it/s]


Conv Autoencoder data shape: (35040, 69, 69, 3)


NameError: name 'batch_size' is not defined

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

num_epochs = 2
for epoch in range(num_epochs):

    for batch_idx, (x, y) in enumerate(train_loader):
        
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(device)
        y = y.to(device)
        break

## Settings

In [ ]:
##########################
### SETTINGS
##########################

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)

# Hyperparameters
random_seed = 123
learning_rate = 0.0005
num_epochs = 50

### Model

In [ ]:
##########################
### MODEL
##########################


class Autoencoder(torch.nn.Module):

    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # calculate same padding:
        # (w - k + 2*p)/s + 1 = o
        # => p = (s(o-1) - w + k)/2
        
        ### ENCODER
        
        # 28x28x1 => 28x28x4
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                      out_channels=4,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      # (1(28-1) - 28 + 3) / 2 = 1
                                      padding=1)
        # 28x28x4 => 14x14x4                              
        self.pool_1 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         # (2(14-1) - 28 + 2) / 2 = 0
                                         padding=0)                                       
        # 14x14x4 => 14x14x8
        self.conv_2 = torch.nn.Conv2d(in_channels=4,
                                      out_channels=8,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      # (1(14-1) - 14 + 3) / 2 = 1
                                      padding=1)                 
        # 14x14x8 => 7x7x8                             
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         # (2(7-1) - 14 + 2) / 2 = 0
                                         padding=0)
        
        ### DECODER
                                         
        # 7x7x8 => 14x14x8                          
        
        ## interpolation
        
        # 14x14x8 => 14x14x8
        self.conv_3 = torch.nn.Conv2d(in_channels=8,
                                      out_channels=4,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      # (1(14-1) - 14 + 3) / 2 = 1
                                      padding=1)
        # 14x14x4 => 28x28x4                            

        ## interpolation
        
        # 28x28x4 => 28x28x1
        self.conv_4 = torch.nn.Conv2d(in_channels=4,
                                      out_channels=1,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      # (1(28-1) - 28 + 3) / 2 = 1
                                      padding=1)
        
    def forward(self, x):
        
        ### ENCODER
        x = self.conv_1(x)
        x = F.leaky_relu(x)
        x = self.pool_1(x)
        x = self.conv_2(x)
        x = F.leaky_relu(x)
        x = self.pool_2(x)
        
        ### DECODER
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = self.conv_3(x)
        x = F.leaky_relu(x)
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = self.conv_4(x)
        x = F.leaky_relu(x)
        x = torch.sigmoid(x)
        return x

    
torch.manual_seed(random_seed)
model = Autoencoder()
model = model.to(device)
    

##########################
### COST AND OPTIMIZER
##########################

cost_fn = torch.nn.BCELoss() # torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [ ]:
##########################
### TRAINING
##########################

epoch_start = 1


torch.manual_seed(random_seed)
model = Autoencoder()
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


################## Load previous
# the code saves the autoencoder
# after each epoch so that in case
# the training process gets interrupted,
# we will not have to start training it
# from scratch
files = os.listdir()

start_time = time.time()
for epoch in range(epoch_start, num_epochs+1):
    
    
    for batch_idx, (x, y) in enumerate(train_loader):

        # don't need labels, only the images (features)
        features = x.to(device)
        
        ### FORWARD AND BACK PROP
        decoded = model(features)
        cost = F.mse_loss(decoded, features)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 500:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch, num_epochs, batch_idx, 
                     len(train_loader), cost))

        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))
        
# Save model
if os.path.isfile('autoencoder_quickdraw-1_i_%d_%s.pt' % (epoch-1, device)):
    os.remove('autoencoder_quickdraw-1_i_%d_%s.pt' % (epoch-1, device))
torch.save(model.state_dict(), 'autoencoder_quickdraw-1_i_%d_%s.pt' % (epoch, device))

## Evaluation

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


model = Autoencoder()
model = model.to(device)
model.load_state_dict(torch.load('autoencoder_quickdraw-1_i_%d_%s.pt' % (num_epochs, device)))
model.eval()
torch.manual_seed(random_seed)

for batch_idx, (x, y) in enumerate(train_loader):
    features = x.to(device)
    decoded = model(features)
    break



##########################
### VISUALIZATION
##########################

n_images = 5

fig, axes = plt.subplots(nrows=2, ncols=n_images, 
                         sharex=True, sharey=True, figsize=(18, 5))
orig_images = features.detach().cpu().numpy()[:n_images]
orig_images = np.moveaxis(orig_images, 1, -1)

decoded_images = decoded.detach().cpu().numpy()[:n_images]
decoded_images = np.moveaxis(decoded_images, 1, -1)


for i in range(n_images):
    for ax, img in zip(axes, [orig_images, decoded_images]):
        ax[i].axis('off')
        ax[i].imshow(img[i].reshape(28, 28), cmap='binary')

In [ ]:
%watermark -iv